In [43]:
import requests
import csv
import os
import pandas as pd
import json
import re
from datetime import datetime
import traceback
today_string = datetime.now().strftime('%Y-%m-%d')

In [2]:
input_path = r"C:\Users\khan32\Documents\WIAGweb2\notebooks\input_files"
output_path = r"C:\Users\khan32\Documents\WIAGweb2\notebooks\output_files"

## Export Csv Personendaten

In [45]:
input_file = f"WIAG-Domherren-DB-Lebensdaten_2024-12-09.csv"
input_path_file = os.path.join(input_path, input_file)
persons_df = pd.read_csv(input_path_file, sep=';')
print(len(persons_df))
persons_df.head()

16219


,id,corpus,givenname,prefix,familyname,displayname,date_of_birth,date_of_death,biographical_dates,summary_offices,GND_ID,GSN,FactGrid_ID,Wikidata_ID,Wikipedia
0,WIAG-Pers-EPISCGatz-03848-001,epc,Ulrich,NaN,NaN,Ulrich,NaN,973,+ 973,"Bischof Augsburg 923-973, Domherr Domstift Aug...",118625284,059-01621-001,Q653546,Q714866,Ulrich_von_Augsburg
1,WIAG-Pers-EPISCGatz-03858-001,epc,Siegfried,NaN,NaN,Siegfried,NaN,1096,+ 1096,"Ernannter Bischof Augsburg 1077-1096, Domherr ...",13805746X,050-06892-001,Q653421,Q64763639,NaN
2,WIAG-Pers-EPISCGatz-04027-001,epc,Norbert,NaN,NaN,Norbert,NaN,NaN,~ 1079/1080,Bischof Chur 1079/1080-1088,NaN,710-00332-001,Q653192,Q35136089,Norbert_von_Hohenwart
3,WIAG-Pers-CANON-91597-001,dreg-can,Sizo,NaN,NaN,Sizo,NaN,NaN,~ um 1099,Domkustos Domstift Augsburg um 1099,NaN,710-00420-001,Q700910,NaN,NaN
4,WIAG-Pers-EPISCGatz-02518-001,epc,Udalschalk,NaN,NaN,Udalschalk,NaN,1202,+ 1202,"Bischof Augsburg 1184-1202, Dompropst Domstift...",138018383,059-01616-001,Q649639,Q68911884,NaN


In [46]:
url = 'https://database.factgrid.de/sparql'
query = """
SELECT ?person ?wiag WHERE {
  ?person wdt:P601 ?wiag.
  ?person wdt:P2 wd:Q7.
}
"""

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
wiag_ids_df = pd.json_normalize(data['results']['bindings'])

len(wiag_ids_df)

13144

In [47]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [48]:
drop_type_columns(wiag_ids_df)
extract_qid(wiag_ids_df, 'person.value')
wiag_ids_df.columns = ['FactGrid_ID', 'old_wiag_id']
wiag_ids_df

,FactGrid_ID,old_wiag_id
0,Q138,WIAG-Pers-CANON-24819-001
1,Q226,WIAG-Pers-EPISCGatz-12636-001
2,Q227,WIAG-Pers-CANON-17813-001
3,Q323,WIAG-Pers-EPISCGatz-12604-001
4,Q347,WIAG-Pers-CANON-17432-001
...,...,...
13139,Q948993,WIAG-Pers-CANON-90831-001
13140,Q948994,WIAG-Pers-CANON-90885-001
13141,Q948995,WIAG-Pers-CANON-90914-001
13142,Q948996,WIAG-Pers-CANON-91436-001


In [49]:
merged_df = wiag_ids_df.merge(persons_df[['FactGrid_ID', 'id']], on='FactGrid_ID')
merged_df

,FactGrid_ID,old_wiag_id,id
0,Q138,WIAG-Pers-CANON-24819-001,WIAG-Pers-CANON-24819-001
1,Q226,WIAG-Pers-EPISCGatz-12636-001,WIAG-Pers-EPISCGatz-12636-001
2,Q227,WIAG-Pers-CANON-17813-001,WIAG-Pers-CANON-17813-001
3,Q323,WIAG-Pers-EPISCGatz-12604-001,WIAG-Pers-EPISCGatz-12604-001
4,Q347,WIAG-Pers-CANON-17432-001,WIAG-Pers-CANON-17432-001
...,...,...,...
9116,Q948993,WIAG-Pers-CANON-90831-001,WIAG-Pers-CANON-90831-001
9117,Q948994,WIAG-Pers-CANON-90885-001,WIAG-Pers-CANON-90885-001
9118,Q948995,WIAG-Pers-CANON-90914-001,WIAG-Pers-CANON-90914-001
9119,Q948996,WIAG-Pers-CANON-91436-001,WIAG-Pers-CANON-91436-001


In [50]:
diff_df = merged_df[merged_df['old_wiag_id'] != merged_df['id']]
diff_df

,FactGrid_ID,old_wiag_id,id


In [51]:
fg_qs_csv = diff_df.copy()
fg_qs_csv.columns = ['qid', '-P601', 'P601']
fg_qs_csv['-P601'] = '"' + fg_qs_csv['-P601'] + '"'
fg_qs_csv['P601'] = '"' + fg_qs_csv['P601'] + '"'
fg_qs_csv.set_index('qid')
fg_qs_csv

,qid,-P601,P601


In [52]:
today_string = datetime.now().strftime('%Y-%m-%d')

# ERROR: Incomplete code below. Please do not run

The code does not generate a file but shows the problems in plain text. This needs to be modified so that is easier to read and automated later.

## Do a cyclic check for all entries in factgrid

In [15]:
from tqdm import tqdm
import aiohttp
import asyncio
import time
import ssl
import copy

In [16]:
print(len(pd.unique(wiag_ids_df['FactGrid_ID'])))
print(len(pd.unique(wiag_ids_df['old_wiag_id'])))
print(len(wiag_ids_df))

13144
13144
13144


In [17]:
wiag_ids_df

,FactGrid_ID,old_wiag_id
0,Q646333,WIAG-Pers-CANON-18832-001
1,Q646334,WIAG-Pers-CANON-18828-001
2,Q646338,WIAG-Pers-CANON-16849-001
3,Q646339,WIAG-Pers-CANON-16678-001
4,Q646340,WIAG-Pers-CANON-16673-001
...,...,...
13139,Q948972,WIAG-Pers-CANON-90763-001
13140,Q948973,WIAG-Pers-CANON-90764-001
13141,Q948974,WIAG-Pers-CANON-90767-001
13142,Q948975,WIAG-Pers-CANON-90768-001


In [29]:
missed = []
new_entries = []
async def get(old_wiag_id, fg_id, session):
    global missed
    wiag_id = old_wiag_id
    try:
        async with session.get(url=f'https://wiag-vocab.adw-goe.de/id/{old_wiag_id}?format=Json') as response:
            data = await response.json()
            wiag_id = data['persons'][0]['wiagId']
            wiag_qid = data['persons'][0]['identifier']['Factgrid'].split('/')[-1]
            if wiag_qid != fg_id:
                print("Wiag linking to different factgrid id", old_wiag_id, fg_id, wiag_qid)
    except KeyError as e:
        ending_str = ''
        if wiag_id != old_wiag_id:
            ending_str = '(' + wiag_id + ')'
            new_entries.append({
                "qid": fg_id,
                "-P601": old_wiag_id,
                "P601": wiag_id,
            })
        else:
            print(f"Factgrid id {fg_id} not present for {old_wiag_id}{ending_str}.")
    except aiohttp.client_exceptions.ContentTypeError as e:
        missed.append([old_wiag_id, fg_id])
    except Exception as e:
        print(old_wiag_id, fg_id)
        print(traceback.format_exc())
        

async def main(urls):
    async with aiohttp.ClientSession() as session:
        chunk_size = 1_000
        for i in range(0, len(urls), chunk_size):
            try:
                await asyncio.gather(*(get(*url, session) for url in urls[i:i+chunk_size]))
                print(f"{i + chunk_size}/{len(urls)} checked. Missed count: {len(missed)}")
                time.sleep(1)
            except ssl.SSLError as e:
                i = i - 1
                print(f"Retrying last batch")
    print(f"Finalized all. Couldn't get data for {len(missed)} entries")

#urls = list(map(lambda x: f'https://wiag-vocab.adw-goe.de/id/{x}?format=Json', wiag_ids_df['old_wiag_id'].to_list()))

##### Note the code cell below can take about 8-10 minutes.

In [30]:
counter = 0
missed = list(zip(list(wiag_ids_df['old_wiag_id']), list(wiag_ids_df['FactGrid_ID'])))
while missed:
    counter += 1
    print(f"Starting attempt #{counter}")
    missed_copy = copy.deepcopy(missed)
    missed = []
    await main(missed_copy)

Starting attempt #1
Factgrid id Q1509 not present for WIAG-Pers-CANON-88455-001.
Factgrid id Q474950 not present for WIAG-Pers-CANON-52288-001.
1000/13144 checked. Missed count: 0
Wiag linking to different factgrid id WIAG-Pers-EPISCGatz-03752-001 Q656456 Q730104
2000/13144 checked. Missed count: 74
3000/13144 checked. Missed count: 111
4000/13144 checked. Missed count: 181
Wiag linking to different factgrid id WIAG-Pers-EPISCGatz-02744-001 Q655623 Q729698
5000/13144 checked. Missed count: 187
Factgrid id Q704412 not present for WIAG-Pers-EPISCGatz-21774-001.
Factgrid id Q700917 not present for WIAG-Pers-CANON-90791-001.
Factgrid id Q701439 not present for WIAG-Pers-CANON-90028-001.
Factgrid id Q701480 not present for WIAG-Pers-CANON-84123-001.
Factgrid id Q701454 not present for WIAG-Pers-CANON-84019-001.
Factgrid id Q701484 not present for WIAG-Pers-CANON-84050-001.
Factgrid id Q701464 not present for WIAG-Pers-CANON-25704-001.
Factgrid id Q701463 not present for WIAG-Pers-CANON-2610

In [53]:
new_df = pd.DataFrame(new_entries)
new_df

,qid,-P601,P601
0,Q701469,WIAG-Pers-CANON-85631-001,WIAG-Pers-CANON-53546-001
1,Q701499,WIAG-Pers-CANON-88617-001,WIAG-Pers-CANON-52044-001
2,Q701443,WIAG-Pers-CANON-88038-001,WIAG-Pers-CANON-49138-001
3,Q701501,WIAG-Pers-CANON-85563-001,WIAG-Pers-CANON-17971-001
4,Q701498,WIAG-Pers-CANON-83974-001,WIAG-Pers-CANON-51356-001
5,Q883666,WIAG-Pers-CANON-81040-001,WIAG-Pers-CANON-53624-001
6,Q883706,WIAG-Pers-CANON-83871-001,WIAG-Pers-CANON-53445-001
7,Q883678,WIAG-Pers-CANON-83507-001,WIAG-Pers-CANON-53455-001
8,Q883725,WIAG-Pers-CANON-84026-001,WIAG-Pers-CANON-53518-001
9,Q883770,WIAG-Pers-CANON-84890-001,WIAG-Pers-CANON-53494-001


In [54]:
final_fg_qs_csv = pd.concat([fg_qs_csv, new_df], ignore_index=True)
final_fg_qs_csv.head()

,qid,-P601,P601
0,Q701469,WIAG-Pers-CANON-85631-001,WIAG-Pers-CANON-53546-001
1,Q701499,WIAG-Pers-CANON-88617-001,WIAG-Pers-CANON-52044-001
2,Q701443,WIAG-Pers-CANON-88038-001,WIAG-Pers-CANON-49138-001
3,Q701501,WIAG-Pers-CANON-85563-001,WIAG-Pers-CANON-17971-001
4,Q701498,WIAG-Pers-CANON-83974-001,WIAG-Pers-CANON-51356-001


This generates a Factgrid file that can be uploaded on to quick statements here https://database.factgrid.de/quickstatements/#/batch. More details to perform this can be found here: TODO add link

In [55]:
final_fg_qs_csv.to_csv(
    os.path.join(
        output_path, 
        f'factgrid_wiag_id_update_{today_string}.csv'
    ), 
    index=False
)